In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/workspace/kaggle_rsna_lumbar_spine/src


In [3]:
# !pip install -qU python-gdcm pydicom pylibjpeg

In [19]:
import os
import cv2
import glob
import gdcm
import pydicom
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from tqdm import tqdm
from joblib import Parallel, delayed
# from pydicom.pixel_data_handlers.util import apply_voi_lut

In [20]:
DATA_PATH = "../input/train_images/"

print('Number of training patients :', len(os.listdir(DATA_PATH)))

Number of training patients : 1975


In [21]:
df = pd.read_csv('../input/train_series_descriptions.csv')

df["orient"] = df["series_description"].apply(lambda x: x.split()[0])
df["weighting"] = df["series_description"].apply(lambda x: x.split()[1])

In [22]:
# dfg = df[["study_id", "orient", "weighting"]].groupby('study_id').agg(list)
# dfg

In [23]:
# (dfg['weighting'].apply(lambda x: len(x))).all()
# (dfg['orient'].apply(lambda x: len(x))).value_counts()

In [24]:
SAVE_FOLDER = "../input/npy2/"
os.makedirs(SAVE_FOLDER, exist_ok=True)

SAVE = True
PLOT = False

In [25]:
from inference.processing import process_2

In [26]:
offsets = []
for study in tqdm(sorted(os.listdir(DATA_PATH))):
    df_study = df[df["study_id"] == int(study)]
    for series in os.listdir(DATA_PATH + study):
        # if os.path.exists(SAVE_FOLDER + f"{study}_{series}.npy"):
        #     continue
        # if series != "813259065":
        #     continue

        df_series = df_study[df_study["series_id"] == int(series)]
        orient, t = df_series["series_description"].values[0].split(" ")

        imgs, df_series = process_2(int(study), int(series), orient, data_path=DATA_PATH, on_gpu=False)
        
#         break
#     break
        offsets.append(
            {
                "study_id": study,
                "series_id": series,
                "frames": df_series.instance_number.values,
            }
        )
            

        if SAVE:
            np.save(SAVE_FOLDER + f"{study}_{series}.npy", imgs)

        if PLOT:
            for i in range(len(imgs)):
                img = imgs[i]
                if not (i % (len(imgs) // 5 + 1)):
                # if i > 40:
                    plt.figure(figsize=(5, 5))
                    plt.imshow(img, cmap="gray")
                    plt.title(
                        f"Study {study} - Series {series} - Frame {i}/{len(imgs)} - Shape {img.shape}"
                    )
                    plt.show()

            # break

#     if PLOT:
#     break

  4%|▍         | 86/1975 [01:32<37:58,  1.21s/it]  

Different shapes: Counter({(640, 640): 26, (608, 608): 24}) resize to (640, 640) - Axial


  7%|▋         | 130/1975 [02:22<31:12,  1.01s/it]

Different shapes: Counter({(640, 640): 26, (608, 608): 23}) resize to (640, 640) - Axial


  7%|▋         | 140/1975 [02:32<30:13,  1.01it/s]

...

 95%|█████████▍| 1874/1975 [32:01<02:18,  1.37s/it]

Different shapes: Counter({(640, 640): 26, (608, 608): 20}) resize to (640, 640) - Axial


 99%|█████████▉| 1965/1975 [33:36<00:08,  1.24it/s]

Different shapes: Counter({(640, 640): 20, (608, 608): 18}) resize to (640, 640) - Axial


100%|██████████| 1975/1975 [33:44<00:00,  1.03s/it]


In [ ]:
df_frames = pd.DataFrame(offsets)
df_frames.to_csv('../input/df_frames.csv', index=False)

In [ ]:
offsets = []
for study in tqdm(sorted(os.listdir(TRAIN_PATH))):
    df_study = df[df["study_id"] == int(study)]
    for series in os.listdir(TRAIN_PATH + study):
        # if os.path.exists(SAVE_FOLDER + f"{study}_{series}.npy"):
        #     continue
        # if series != "813259065":
        #     continue

        df_series = df_study[df_study["series_id"] == int(series)]
        orient, t = df_series["series_description"].values[0].split(" ")

        folder = TRAIN_PATH + f"{study}/{series}/"
        files = os.listdir(folder)
        files.sort(key=lambda x: int(x[:-4]))

        imgs = {}
        for frame, file in enumerate(files):
            dicom = pydicom.dcmread(folder + file)

            # Retrieve frame order
            pos = int(file.split("/")[-1][:-4])
            # print(dicom[(0x20, 0x32)])
            if orient == "Axial":
                pos = -dicom[(0x20, 0x32)].value[-1]  
            else:  # Sagittal
                pos = dicom[(0x20, 0x32)].value[0]

            img = dicom.pixel_array

            if dicom.PhotometricInterpretation == "MONOCHROME1":
                print("inv")
                img = 1 - img

            try:
                _ = imgs[pos]
                print(f"Pos {pos} is already in keys")
                imgs[pos + 0.1] = img  # pos is the same, offset by 0.1
            except KeyError:
                imgs[pos] = img

        assert len(imgs) == len(files), "Missing frames!"

        order = np.argsort(list(imgs.keys()))
        offsets.append(
            {
                "study_id": study,
                "series_id": series,
                "frames": [int(f[:-4]) for f in np.array(files)[order]],
            }
        )

        try:
            imgs = np.array([img for k, img in sorted(imgs.items())])
        except:
            imgs = [img for k, img in sorted(imgs.items())]

            shapes = Counter([img.shape for img in imgs])
            shape = shapes.most_common()[0][0]
            print("Different shapes:", shapes, f"resize to {shape}")

            imgs = np.array(
                [cv2.resize(img, shape) if img.shape != shape else img for img in imgs]
            )

        if SAVE:
            np.save(SAVE_FOLDER + f"{study}_{series}.npy", imgs)

        if PLOT:
            for i in range(len(imgs)):
                img = imgs[i]
                # if not (i % (len(imgs) // 5 + 1)):

                if i > 40:
                    plt.figure(figsize=(5, 5))
                    plt.imshow(img, cmap="gray")
                    plt.title(
                        f"Study {study} - Series {series} - Frame {i}/{len(imgs)} - Shape {img.shape}"
                    )
                    plt.show()

            # break

    if PLOT:
        break

In [14]:
df_frames = pd.DataFrame(offsets)
df_frames.to_csv('../input/df_frames.csv', index=False)

In [ ]:
# folder = "../input/train_images/1644528034/2456924377/"
# folder = "../input/train_images/2794192602/2916494105/"
# folder = "../input/train_images/2794192602/1829533928/"
# folder = "../input/train_images/2794192602/1843512620/"
# folder = "../input/train_images/3303545110/304087230/"
# folder = "../input/train_images/642715533/3541751675/"

# files = os.listdir(folder)
# files.sort(key=lambda x:int(x[:-4]))
# files

Done ! 